# Sorting the conferences via "date" field

In [5]:
import datetime,json
import pandas as pd

## Sorting with pandas

In [10]:
confs = pd.read_json("_data/Conference.json")
confs.dropna(inplace=True)
confs

,title,location,date,url
1,HEP Software Training with HSF/IRIS-HEP,"New Perspectives Meeting (Fermilab, Batavia, I...",2023-06-27,https://indico.fnal.gov/event/59506/contributi...
2,Trigger Development for Emerging Jet Analysis ...,"FNAL 56st Annual Users Meeting (Fermilab, Bata...",2023-06-28,https://indico.fnal.gov/event/59656/contributi...
3,Teaching Python the Sustainable Way: Lessons L...,PyHEP 2022 Workshop (Virtual),2022-09-12,https://indico.cern.ch/event/1150631/contribut...
4,PyROOT tutorial experience from SWC Workshop,12th ROOT Users' Workshop (Virtual),2022-05-10,https://indico.fnal.gov/event/23628/contributi...
5,New Trigger Studies for Emerging Jets at CMS E...,"PRIMS/JTM Conference (Humacao, Puerto Rico)",2022-04-09,/assets/pdfs/PRISM_BetterPoster_EMJ.pdf
6,Recent Progress in ML for Tracker DQM,DPF Conference (Virtual),2021-07-12,https://indico.cern.ch/event/1034469/contribut...
7,Machine Learning in DQM at CMS Experiment,"Physcon Conference (Rhode Island, USA)",2019-11-13,/assets/pdfs/Physcon_Poster.pdf
8,Machine Learning and Deep Neural Networks at C...,Physics symposium (University of Puerto Rico M...,2019-05-24,http://charma.uprm.edu/~malik/Undergrad_Summer...
9,Machine Learning in DQM at CMS Experiment,"PRISM/JTM Conference (Mayagüez, Puerto Rico)",2019-05-04,/assets/pdfs/ML4DQM_PRISM_2019_Talk.pdf
10,Using ML techniques for DQM at CMS,ML Hackathon (University of Puerto Rico Mayagüez),2019-04-25,https://indico.cern.ch/event/809812/contributi...


In [18]:
import pprint

In [19]:
pprint.pprint(confs.to_json(orient='records',date_format="iso",))

('[{"title":"HEP Software Training with HSF\\/IRIS-HEP","location":"New '
 'Perspectives Meeting (Fermilab, Batavia, IL, '
 'USA)","date":"2023-06-27T00:00:00.000","url":"https:\\/\\/indico.fnal.gov\\/event\\/59506\\/contributions\\/269987\\/"},{"title":"Trigger '
 'Development for Emerging Jet Analysis (Poster)","location":"FNAL 56st Annual '
 'Users Meeting (Fermilab, Batavia, IL, '
 'USA)","date":"2023-06-28T00:00:00.000","url":"https:\\/\\/indico.fnal.gov\\/event\\/59656\\/contributions\\/268969\\/"},{"title":"Teaching '
 'Python the Sustainable Way: Lessons Learned at HSF '
 'Training","location":"PyHEP 2022 Workshop '
 '(Virtual)","date":"2022-09-12T00:00:00.000","url":"https:\\/\\/indico.cern.ch\\/event\\/1150631\\/contributions\\/5014278\\/"},{"title":"PyROOT '
 'tutorial experience from SWC Workshop","location":"12th ROOT Users\' '
 'Workshop '
 '(Virtual)","date":"2022-05-10T00:00:00.000","url":"https:\\/\\/indico.fnal.gov\\/event\\/23628\\/contributions\\/240752\\/"},{"title

In [115]:
result = confs.sort_values("date",ascending=False).to_json(orient="records",date_format='iso')
parsed = json.loads(result)
json.dump(parsed,open("_data/sorted_confs.json",'w'), indent=4)  

## Creating the bibfile from inspire

In [1]:
import requests
url = 'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=a Guillermo.Fidalgo.1&format=bibtex'
inspire = requests.get(url).text
# response = requests.get('https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=Guillermo%20Fidalgo')
# url = response.json()['links']['bibtex']
# print(url)
# pprint.pprint(json_out['links'])
# inspire = requests.get(url).text
print(inspire)

@article{CMS:2024gxp,
    author = "Hayrapetyan, Aram and others",
    collaboration = "CMS",
    title = "{Search for new physics with emerging jets in proton-proton collisions at $\sqrt{s}$ = 13 TeV}",
    eprint = "2403.01556",
    archivePrefix = "arXiv",
    primaryClass = "hep-ex",
    reportNumber = "CMS-EXO-22-015, CERN-EP-2024-049",
    month = "3",
    year = "2024"
}

@inproceedings{Malik:2022bdz,
    author = "Malik, Sudhir and Karadzhinova-Ferrer, Aneliya and Hogan, Julie and Bray, Rachel and Kamalieddin, Rami and Flood, Kevin and El-Zant, Amr and Fidalgo, Guillermo and Bruhwiler, David and Bellis, Matt",
    title = "{Facilitating Non-HEP Career Transition}",
    booktitle = "{Snowmass 2021}",
    eprint = "2203.11665",
    archivePrefix = "arXiv",
    primaryClass = "physics.ed-ph",
    month = "3",
    year = "2022"
}

@article{Bardeen:2022zwf,
    author = "Bardeen, Marge G. and others",
    title = "{Particle Physics Outreach to K-12 Schools and Opportunities in Under

In [3]:
zenodo = r"""
@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}
"""
print(zenodo)

@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}



Making it into a script

In [109]:
with open("bibfile.bib",'w') as f:
    f.write(inspire)
    f.write(zenodo)

In [160]:
%%writefile createbib.py 
import requests
response = requests.get('https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=Guillermo%20Fidalgo')
url = response.json()['links']['bibtex']
inspire = requests.get(url).text

zenodo = r"""
@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}
"""
with open("_data/bibfile.bib",'w') as f:
    f.write(inspire)
    f.write(zenodo)

Writing createbib.py


In [162]:
! python createbib.py

## Making tests with Pandoc

Just run
```shell
pandoc -C _data/Publications.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/myPubs.md
```

In [2]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/myPubs.md

pandoc: _data/Publications.bib: openBinaryFile: does not exist (No such file or directory)


In [1]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
-o deleteME.md

[WARNING] Could not convert TeX math \sqrt{s}, rendering as TeX
[WARNING] Could not convert TeX math \sqrt{s}, rendering as TeX


In [ ]:
!pandoc --help

pandoc [OPTIONS] [FILES]
  -f FORMAT, -r FORMAT  --from=FORMAT, --read=FORMAT                    
  -t FORMAT, -w FORMAT  --to=FORMAT, --write=FORMAT                     
  -o FILE               --output=FILE                                   
                        --data-dir=DIRECTORY                            
  -M KEY[:VALUE]        --metadata=KEY[:VALUE]                          
                        --metadata-file=FILE                            
  -d FILE               --defaults=FILE                                 
                        --file-scope                                    
                        --sandbox                                       
  -s                    --standalone                                    
                        --template=FILE                                 
  -V KEY[:VALUE]        --variable=KEY[:VALUE]                          
                        --wrap=auto|none|preserve                       
                        --